In [ ]:
%cd /content/drive/MyDrive/Research/datasets/hf_datasets/

/content/drive/MyDrive/Research/datasets/hf_datasets


In [ ]:
!pip install datasets --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


## Load dataset

In [ ]:
# it loads entire dataset first
from datasets import load_dataset
from google.colab import userdata
token=userdata.get('HUGGING_FACE_BEARER')

# Load nepberta configuration
nepberta_data = load_dataset("Aananda-giri/nepali_llm_datasets", name="nepberta", token=token)

## Tokenize

In [ ]:
import pandas as pd
import pyarrow as pa
from transformers import PreTrainedTokenizerFast

# Load the tokenizer
tokenizer = PreTrainedTokenizerFast.from_pretrained("Aananda-giri/NepaliBPE")

train_data = nepberta_data['train']
test_data = nepberta_data['test']

# Initialize lists to collect input_ids and target_ids
input_ids_list = []
target_ids_list = []

import pyarrow.parquet as pq
import pandas as pd


import pyarrow.parquet as pq
import pandas as pd
import pyarrow as pa

def prepare_dataset(txt, tokenizer, max_length, stride, index):
    input_ids = []
    target_ids = []
    save_file = f"pre_tokenized/nepberta_data_{index}.parquet"
    token_ids = tokenizer.encode(txt)

    for i in range(0, len(token_ids) - max_length, stride):
        input_chunk = token_ids[i:i + max_length]
        target_chunk = token_ids[i + 1: i + max_length + 1]

        input_ids.append(input_chunk)
        target_ids.append(target_chunk)

    print('... tokenized', end='')
    df = pd.DataFrame({
      "input_ids": input_ids,
      "target_ids": target_ids
    })
    print('... df created', end='')

    # Convert DataFrame to pyarrow Table
    table = pa.Table.from_pandas(df)

    # # Try to read existing Parquet file
    # try:
    #     existing_table = pq.read_table(save_file)
    #     # Concatenate existing table with new table
    #     table = pa.concat_tables([existing_table, table])
    # except FileNotFoundError:
    #     print('file not found')  # File does not exist, so just write the new table

    # Write table to Parquet file
    pq.write_table(table, save_file, compression='snappy')
    print(':...saved!')

# Writing each batch to the Parquet file
# i=0
# for data in train_data:
#     # combine two chunks
#     i+=1;print(i, end='')
#     prepare_dataset(data['text'], tokenizer, max_length=1024, stride=1024, index=i)
i=len(train_data)
for data in test_data:
    # combine two chunks
    i+=1;print(i, end='')
    prepare_dataset(data['text'], tokenizer, max_length=1024, stride=1024, index=i)

185... tokenized... df created:...saved!
186... tokenized... df created:...saved!
187... tokenized... df created:...saved!
188... tokenized... df created:...saved!
189... tokenized... df created:...saved!
190... tokenized... df created:...saved!
191... tokenized... df created:...saved!
192... tokenized... df created:...saved!
193... tokenized... df created:...saved!
194... tokenized... df created:...saved!
195... tokenized... df created:...saved!
196... tokenized... df created:...saved!
197... tokenized... df created:...saved!
198... tokenized... df created:...saved!
199... tokenized... df created:...saved!
200... tokenized... df created:...saved!
201... tokenized... df created:...saved!
202... tokenized... df created:...saved!
203... tokenized... df created:...saved!
204... tokenized... df created:...saved!
205... tokenized... df created:...saved!
206... tokenized... df created:...saved!
207... tokenized... df created:...saved!
208... tokenized... df created:...saved!
209... tokenized

In [ ]:
len(train_data) + len(test_data)

230

In [ ]:
# Merge the .parquet files generated
import pyarrow.parquet as pq
import glob
import pyarrow as pa

# Specify the input folder and output file
input_folder = 'pre_tokenized'
output_file = 'nepberta.parquet'

# Get a list of all Parquet files in the input folder
parquet_files = glob.glob(f'{input_folder}/*.parquet')
print(len(parquet_files))

# Write each file to the output file one at a time
writer = None
i=0
for file in parquet_files:
    i+=1;print(i)
    table = pq.read_table(file)
    if writer is None:
        # Initialize the writer with the schema of the first file
        writer = pq.ParquetWriter(output_file, table.schema)
    writer.write_table(table)

# Close the writer
if writer:
    writer.close()


1
1


In [ ]:
# Delete the original files (after merging)
import os
for file in parquet_files:
    try:
        os.remove(file)
        print(f"Deleted file: {file}")
    except OSError as e:
        print(f"Error deleting file: {file} - {e.strerror}")

## upload to huggingface

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

from google.colab import userdata


# Upload all the content from the local folder to your remote Space.
# By default, files are uploaded at the root of the repo
api.upload_folder(
    folder_path="./pre_tokenized",
    path_in_repo="pre_tokenized", # Upload to a specific folder
    repo_id="Aananda-giri/nepali_llm_datasets",
    repo_type="dataset",
    token=userdata.get('HF_TOKEN')
)

nepberta.parquet:   0%|          | 0.00/3.15G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Aananda-giri/nepali_llm_datasets/commit/aab4761afa9abd3f44bde4a26a5314c926f7ac39', commit_message='Upload folder using huggingface_hub', commit_description='', oid='aab4761afa9abd3f44bde4a26a5314c926f7ac39', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Aananda-giri/nepali_llm_datasets', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Aananda-giri/nepali_llm_datasets'), pr_revision=None, pr_num=None)

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) 
Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# Delete files uploaded at ~/ location in huggingface
# !huggingface-cli repo-files Aananda-giri/nepali_llm_datasets delete pre-tokenize.ipynb --repo-type dataset
# !huggingface-cli repo-files Aananda-giri/nepali_llm_datasets delete pre_tokenized/nepberta.parquet --repo-type dataset

Files correctly deleted from repo. Commit: https://huggingface.co/datasets/Aananda-giri/nepali_llm_datasets/commit/224c8917e09f74fb37fa4f953ab344c30fcf3e0d.


## update README.md

refer to [../preprocess.ipynb](../preprocess.ipynb) to update README.md


## load dataset from huggingface (problem/solution)

#### problem
this code (loading parquet dataset locally) works but uploading same file to huggingface and loading using below code is giving error: UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe0 in position 7: invalid continuation byte

```
%cd /content/drive/MyDrive/Research/datasets/hf_datasets/
!pip install datasets --quiet

from datasets import load_dataset
ds = load_dataset('parquet', data_files = 'nepberta.parquet')
```



In [ ]:
dataset = load_dataset("aananda-giri/nepali_llm_datasets", name="pre_tokenized", token=token)
dataset

Resolving data files:   0%|          | 0/184 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/46 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetGenerationError: An error occurred while generating the dataset

#### Solution1 (not working): Save this dataset to huggingface and try loading that instead

In [ ]:
# ```
%cd /content/drive/MyDrive/Research/datasets/hf_datasets/
!pip install datasets --quiet

from datasets import load_dataset
ds = load_dataset('parquet', data_files = 'nepberta.parquet')

# above code works but uploading same file to huggingface and loading using below code is giving error: UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe0 in position 7: invalid continuation byte
# Save this dataset to huggingface and try loading that instead

/content/drive/MyDrive/Research/datasets/hf_datasets


Generating train split: 0 examples [00:00, ? examples/s]

Loading dataset shards:   0%|          | 0/26 [00:00<?, ?it/s]

In [ ]:
# (works but lets not split the dataset)Split the dataset into 90% train and 10% test
split_data = ds['train'].train_test_split(test_size=0.1, seed=42)

from datasets import DatasetDict

# Organize the splits into a DatasetDict
dataset_dict = DatasetDict({
    'train': split_data['train'],
    'test': split_data['test']
})

# from google.colab import userdata

# # Push the dataset to Hugging Face
# dataset_dict.push_to_hub("aananda-giri/pre_tokenized", token=userdata.get('HUGGING_FACE_BEARER'))

import os

# Create the local folder structure
os.makedirs("pre_tokenized", exist_ok=True)

# Save each split to the pre_tokenized folder as Parquet files
dataset_dict['train'].to_parquet("pre_tokenized/nepberta_train.parquet")
dataset_dict['test'].to_parquet("pre_tokenized/nepberta_test.parquet")

#### solution2 (not working: use solution1 but save single parquet file instead of train and test.par

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'target_ids'],
        num_rows: 778572
    })
})

In [ ]:
# (not workiing but lets not split the dataset)Split the dataset into 90% train and 10% test

from datasets import DatasetDict
import os

# Create the local folder structure
os.makedirs("pre_tokenized", exist_ok=True)

# Save each split to the pre_tokenized folder as Parquet files
ds['train'].to_parquet("pre_tokenized/nepberta.parquet")


Creating parquet from Arrow format:   0%|          | 0/779 [00:00<?, ?ba/s]

12762352224

In [ ]:
ds['train'][0].keys()

dict_keys(['input_ids', 'target_ids'])

In [ ]:
!huggingface-cli login
!huggingface-cli repo-files Aananda-giri/nepali_llm_datasets delete pre_tokenized/nepberta.parquet --repo-type dataset


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) 
Token is valid (permission: write).
The token `colab_write` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/hu

#### solution3 (working)
* load parquet files remotely as mentioned [here](https://huggingface.co/datasets/Aananda-giri/nepali_llm_datasets/resolve/main/pre_tokenized/nepberta.parquet)

* need to set environment variable: HF_TOKEN as:

`os.environ['HF_TOKEN']`

In [ ]:
# base_url = "https://huggingface.co/datasets/Aananda-giri/nepali_llm_datasets/resolve/main/pre_tokenized/"
# data_files = {"train": base_url + "nepberta_train.parquet", "test": base_url + "nepberta_test.parquet"}
# dataset = load_dataset("parquet", data_files=data_files)
# print(dataset)

# Alternatively: download non-splitted datasets
base_url = "https://huggingface.co/datasets/Aananda-giri/nepali_llm_datasets/resolve/main/pre_tokenized/"
data_files = {"train": base_url + "nepberta.parquet"}
dataset = load_dataset("parquet", data_files=data_files, split="train")
print(dataset)

# and split it later
dataset = dataset.train_test_split(train_size=0.9, seed=42)
print(dataset)

Loading dataset shards:   0%|          | 0/26 [00:00<?, ?it/s]

Dataset({
    features: ['input_ids', 'target_ids'],
    num_rows: 778572
})
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'target_ids'],
        num_rows: 700714
    })
    test: Dataset({
        features: ['input_ids', 'target_ids'],
        num_rows: 77858
    })
})


In [ ]:
a['test']

Dataset({
    features: ['input_ids', 'target_ids'],
    num_rows: 77858
})

In [ ]:
train_dataset

'train'

#### probable solution 4

**in prepare_dataset function, use:**
```
input_ids.append(np.array(input_chunk))
target_ids.append(np.array(target_chunk))
```
**instead of**
```
input_ids.append(input_chunk)
target_ids.append(target_chunk)
```

#### information about dataset

In [ ]:
from datasets import get_dataset_config_names

configs = get_dataset_config_names("aananda-giri/nepali_llm_datasets")
print(configs)

Resolving data files:   0%|          | 0/184 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/46 [00:00<?, ?it/s]

['nepberta', 'scrapy_engine', 'pre_tokenized']


In [ ]:
from datasets import load_dataset_builder
ds_builder = load_dataset_builder("aananda-giri/nepali_llm_datasets", name="pre_tokenized")

print(ds_builder.info.description)

print(ds_builder.info.features)

Resolving data files:   0%|          | 0/184 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/46 [00:00<?, ?it/s]


None


In [ ]:
# Split the dataset (80% train, 20% test)
split_data = dataset['train'].train_test_split(test_size=0.2, seed=42)

train_data = split_data['train']
test_data = split_data['test']

print("Train size:", len(train_data))
print("Test size:", len(test_data))

# Sebastian DataLoader with pre_tokenized dataset from huggingface

### Using pre-tokenized dataset with dataloader

In [ ]:
from torch.utils.data import DataLoader
from datasets import load_dataset
import torch


def create_dataloader_v2(batch_size=4, shuffle=True, drop_last=True, num_workers=0, train_ratio=.9):
    '''
    modified.
    * dont need text data as input
    * dont need max_length and stride as input : they were set during preparing tokenized_datasets
    '''
    # Download the whole dataset
    base_url = "https://huggingface.co/datasets/Aananda-giri/nepali_llm_datasets/resolve/main/pre_tokenized/"
    data_files = {"train": base_url + "nepberta.parquet"}
    dataset = load_dataset("parquet", data_files=data_files, split="train")

    print(dataset)

    # and split it later
    dataset = dataset.train_test_split(train_size=train_ratio, seed=42)
    # Convert Hugging Face Dataset to PyTorch tensors (we can directly use the dataset as it is already in the correct format)
    dataset.set_format(type="torch", columns=["input_ids", "target_ids"])  # Directly set columns to torch tensors



    # Define the custom collate_fn function
    def collate_fn(batch):
        # Extract the 'input_ids' and 'target_ids' from the batch and return them as a list of tensors
        input_ids = [item['input_ids'] for item in batch]
        target_ids = [item['target_ids'] for item in batch]

        # Convert to tensors (if not already)
        input_ids_tensor = torch.stack(input_ids)
        target_ids_tensor = torch.stack(target_ids)

        return [input_ids_tensor, target_ids_tensor]



    # Creating the DataLoader for the 'train' split of the dataset with the custom collate_fn
    train_loader = DataLoader(
        dataset['train'],
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers,
        collate_fn=collate_fn
    )

    test_loader =  DataLoader(
        dataset['test'],
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers,
        collate_fn=collate_fn
    )

    return train_loader, test_loader

train_loader, test_loader = create_dataloader_v2(
    batch_size=4,
    shuffle=True,
    drop_last=True,
    num_workers=0,
    train_ratio=.9
)
train_loader

Loading dataset shards:   0%|          | 0/26 [00:00<?, ?it/s]

Dataset({
    features: ['input_ids', 'target_ids'],
    num_rows: 778572
})


In [ ]:
print(len(train_loader))

for input_batch, target_batch in train_loader:
    print(f"len(input_batch):{len(input_batch)}")
    print(f"len(input_batch[0]):{len(input_batch[0])}")
    print(f"input_batch.numel(): {input_batch.numel()}")
    print(f"type(input_batch): {type(input_batch)}", end='\n\n')

    print(f"len(target_batch): {len(target_batch)}")
    print(f"len(input_batch[0]):{len(target_batch[0])}")
    print(f"target_batch.numel(): {target_batch.numel()}")
    print(f"type(target_batch): {type(target_batch)}")
    break


175178
len(input_batch):4
len(input_batch[0]):1024
input_batch.numel(): 4096
type(input_batch): <class 'torch.Tensor'>

len(target_batch): 4
len(input_batch[0]):1024
target_batch.numel(): 4096
type(target_batch): <class 'torch.Tensor'>


### comparision with original dataloader by sebastian

In [ ]:
from torch.utils.data import Dataset, DataLoader
from transformers import PreTrainedTokenizerFast

# use streaming=True to avoid downloading entire dataset
nepberta_train = load_dataset("Aananda-giri/nepali_llm_datasets", name="nepberta", streaming=True)['train']
text_data = next(iter(nepberta_train))['text']
print(f"text_chunk length: {len(text_data)}")

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # modified.
        # token_ids = tokenizer.encode(txt, allowed_special={'<|endoftext|>'})
        token_ids = tokenizer.encode(txt)

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]


def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True, num_workers=0):
    # -------------------------------
    # Modofy tokenizer
    # -------------------------------
    # modified. tokenizer initialization
    # tokenizer = tiktoken.get_encoding("gpt2")
    # Load the tokenizer
    tokenizer = PreTrainedTokenizerFast.from_pretrained("Aananda-giri/NepaliBPE")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)

    return dataloader

train_ratio = 0.9
split_idx = int(train_ratio * len(text_data))
train_loader = create_dataloader_v1(
        text_data[:split_idx],
        batch_size=4,
        max_length=1024,
        stride=1024,
        drop_last=True,
        shuffle=True,
        num_workers=0
    )
train_loader

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

text_chunk length: 19626198


tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.53M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.00 [00:00<?, ?B/s]

In [ ]:
print(len(train_loader))

for input_batch, target_batch in train_loader:
    print(f"len(input_batch):{len(input_batch)}")
    print(f"len(input_batch[0]):{len(input_batch[0])}")
    print(f"input_batch.numel(): {input_batch.numel()}")
    print(f"type(input_batch): {type(input_batch)}", end='\n\n')

    print(f"len(target_batch): {len(target_batch)}")
    print(f"len(input_batch[0]):{len(target_batch[0])}")
    print(f"target_batch.numel(): {target_batch.numel()}")
    print(f"type(target_batch): {type(target_batch)}")
    break


771
len(input_batch):4
len(input_batch[0]):1024
input_batch.numel(): 4096
type(input_batch): <class 'torch.Tensor'>

len(target_batch): 4
len(input_batch[0]):1024
target_batch.numel(): 4096
type(target_batch): <class 'torch.Tensor'>


*  using pre-tokenized, we got the similar result as given by sebastian original dataloader

# also replace `create_dataloaders` function from  pretraining_bells_n_whistles file

In [ ]:
def create_dataloaders(train_ratio, batch_size, num_workers=0):
    '''
    we dont need
    text_data
    max_length, stride

    '''
    train_loader, val_loader = create_dataloader_v2(
        batch_size=batch_size,
        shuffle=True,
        drop_last=True,
        num_workers=num_workers,
        train_ratio=train_ratio
    )
    return train_loader, val_loader

# Example: append to parquet

In [ ]:
import pyarrow as pa
import pyarrow.parquet as pq

# Sample data loader simulation
# Let's create a list of dictionaries to simulate batches of data with text to be tokenized
# In practice, each `data` entry could represent a batch, for simplicity here, we'll use one item per batch

data_loader = [
    {'text': "The quick brown fox"},
    {'text': "jumps over the lazy dog"},
    {'text': "and runs away quickly"},
    {'text': "The sun sets behind the mountains"},
    {'text': "A quick brown dog jumps over the lazy fox"},
]

# Define a simple tokenization function for demonstration
def get_tokenized(text):
    # Simulating tokenization to input_ids and target_ids (just for example)
    input_ids = [ord(char) for char in text]  # Convert each character to ASCII value
    target_ids = [ord(char) + 1 for char in text]  # Shift ASCII values by 1
    return input_ids, target_ids

# Define the schema based on `input_ids` and `target_ids` being lists of integers
schema = pa.schema([
    ('input_ids', pa.list_(pa.int64())),
    ('target_ids', pa.list_(pa.int64()))
])

# Path for the Parquet file
parquet_file_path = "nepberta_data.parquet"

# Writing each batch to the Parquet file
with pq.ParquetWriter(parquet_file_path, schema) as writer:
    for data in data_loader:
        # Tokenize the text to get input_ids and target_ids
        input_ids, target_ids = get_tokenized(data['text'])

        # because dataloader seeks same sized data in a batch
        input_ids = input_ids[:18]
        target_ids = target_ids[:18]

        # Create a record batch for the current data
        batch = pa.RecordBatch.from_arrays([pa.array([input_ids]), pa.array([target_ids])], schema=schema)

        # Write batch to Parquet file
        writer.write_table(pa.Table.from_batches([batch]))

parquet_file_path


## upload to huggingface

In [ ]:
%cd /content/drive/MyDrive/Research/datasets/hf_datasets

In [ ]:
from huggingface_hub import HfApi, Repository
import os
import shutil

# Set your dataset repo name and Hugging Face username
username = "Aananda-giri"  # Replace with your Hugging Face username
repo_name = "nepali_llm_datasets"  # Name for the new dataset repository
repo_id = f"{username}/{repo_name}"  # Full repository ID

# Authenticate
api = HfApi()
# token = "token"  # Replace with your Hugging Face token

# Local dataset directory
local_dir = "./"  # Adjust to the path of the dataset folder

# Create the repository if it does not exist
api.create_repo(repo_id, token=token, repo_type="dataset", exist_ok=True)

In [ ]:
# Local dataset directory
local_dir = "./"  # Path to your dataset folder

# Upload all files and folders recursively
for root, dirs, files in os.walk(local_dir):
    for file in files:
        file_path = os.path.join(root, file)
        # Upload file to the specified repo with the same directory structure
        repo_file_path = os.path.relpath(file_path, local_dir)
        api.upload_file(
            path_or_fileobj=file_path,
            path_in_repo=repo_file_path,
            repo_id=repo_id,
            repo_type="dataset",
            token=token
        )

print(f"Dataset uploaded to https://huggingface.co/datasets/{repo_id}")

# Example: upload to huggingface

In [ ]:
from datasets import load_dataset
from google.colab import userdata
# Load Parquet file as a Dataset
dataset = load_dataset("parquet", data_files="*.parquet")

# Push to Hugging Face Hub
dataset.push_to_hub("Aananda-giri/tokenized_dataset_parquet", token=userdata.get('HUGGING_FACE_BEARER'))


Generating train split: 0 examples [00:00, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/310 [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/datasets/Aananda-giri/tokenized_dataset_parquet/commit/677a7569e167fdce921e3a2150342fb407025f3e', commit_message='Upload dataset', commit_description='', oid='677a7569e167fdce921e3a2150342fb407025f3e', pr_url=None, pr_revision=None, pr_num=None)

 # Example: load from huggingface

In [ ]:
from datasets import load_dataset

ds = load_dataset("Aananda-giri/tokenized_dataset_parquet")
ds
'''
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'target_ids'],
        num_rows: 3
    })
})
'''

"\nDatasetDict({\n    train: Dataset({\n        features: ['input_ids', 'target_ids'],\n        num_rows: 3\n    })\n})\n"

In [ ]:
for data in ds['train']:
    print(data)

{'input_ids': [84, 104, 101, 32, 113, 117, 105, 99, 107, 32, 98, 114, 111, 119, 110, 32, 102, 111], 'target_ids': [85, 105, 102, 33, 114, 118, 106, 100, 108, 33, 99, 115, 112, 120, 111, 33, 103, 112]}
{'input_ids': [106, 117, 109, 112, 115, 32, 111, 118, 101, 114, 32, 116, 104, 101, 32, 108, 97, 122], 'target_ids': [107, 118, 110, 113, 116, 33, 112, 119, 102, 115, 33, 117, 105, 102, 33, 109, 98, 123]}
{'input_ids': [97, 110, 100, 32, 114, 117, 110, 115, 32, 97, 119, 97, 121, 32, 113, 117, 105, 99], 'target_ids': [98, 111, 101, 33, 115, 118, 111, 116, 33, 98, 120, 98, 122, 33, 114, 118, 106, 100]}
{'input_ids': [84, 104, 101, 32, 115, 117, 110, 32, 115, 101, 116, 115, 32, 98, 101, 104, 105, 110], 'target_ids': [85, 105, 102, 33, 116, 118, 111, 33, 116, 102, 117, 116, 33, 99, 102, 105, 106, 111]}
{'input_ids': [65, 32, 113, 117, 105, 99, 107, 32, 98, 114, 111, 119, 110, 32, 100, 111, 103, 32], 'target_ids': [66, 33, 114, 118, 106, 100, 108, 33, 99, 115, 112, 120, 111, 33, 101, 112, 104, 

In [ ]:
for data in ds['train']:
  print(len(data['input_ids']), len(data['target_ids']))
  # print(data)

18 18
18 18
18 18
18 18
18 18


# Example: Our dataloader with huggingface datasets

In [ ]:
from torch.utils.data import DataLoader
from datasets import load_dataset
import torch

# Load the dataset from Hugging Face
ds = load_dataset("Aananda-giri/tokenized_dataset_parquet")

# Define the custom collate_fn function
def collate_fn(batch):
    # Extract the 'input_ids' and 'target_ids' from the batch and return them as a list of tensors
    input_ids = [item['input_ids'] for item in batch]
    target_ids = [item['target_ids'] for item in batch]

    # Convert to tensors (if not already)
    input_ids_tensor = torch.stack(input_ids)
    target_ids_tensor = torch.stack(target_ids)

    return [input_ids_tensor, target_ids_tensor]

# Convert Hugging Face Dataset to PyTorch tensors (we can directly use the dataset as it is already in the correct format)
ds.set_format(type="torch", columns=["input_ids", "target_ids"])  # Directly set columns to torch tensors

# Creating the DataLoader for the 'train' split of the dataset with the custom collate_fn
dataloader = DataLoader(
    ds["train"],
    batch_size=4,
    shuffle=True,
    drop_last=True,
    num_workers=0,
    collate_fn=collate_fn
)

# Example of accessing one batch
for batch in dataloader:
    print(batch)
    break


[tensor([[106, 117, 109, 112, 115,  32, 111, 118, 101, 114,  32, 116, 104, 101,
          32, 108,  97, 122],
        [ 97, 110, 100,  32, 114, 117, 110, 115,  32,  97, 119,  97, 121,  32,
         113, 117, 105,  99],
        [ 84, 104, 101,  32, 115, 117, 110,  32, 115, 101, 116, 115,  32,  98,
         101, 104, 105, 110],
        [ 84, 104, 101,  32, 113, 117, 105,  99, 107,  32,  98, 114, 111, 119,
         110,  32, 102, 111]]), tensor([[107, 118, 110, 113, 116,  33, 112, 119, 102, 115,  33, 117, 105, 102,
          33, 109,  98, 123],
        [ 98, 111, 101,  33, 115, 118, 111, 116,  33,  98, 120,  98, 122,  33,
         114, 118, 106, 100],
        [ 85, 105, 102,  33, 116, 118, 111,  33, 116, 102, 117, 116,  33,  99,
         102, 105, 106, 111],
        [ 85, 105, 102,  33, 114, 118, 106, 100, 108,  33,  99, 115, 112, 120,
         111,  33, 103, 112]])]
